В папке с программой должен лежать файл quora_question_pairs_rus.csv

In [1]:
from math import log
import numpy as np
import pymorphy2 as pm2 
pmm = pm2.MorphAnalyzer() 
import re
import json
from operator import itemgetter
import timeit
import time

In [2]:
def clean_text(text):
    text = re.sub(r'[^\w\s]','',text) 
    text = [pmm.normal_forms(x)[0] for x in text.split()] 
    return text

In [3]:
import csv
with open('quora_question_pairs_rus.csv', 'r', encoding='utf-8') as q:
    str_corpus = csv.reader(q)
    file = list(str_corpus)

In [4]:
file = file[1:]

In [5]:
queries = []
for i in file:
    queries.append(i[1])

In [7]:
d_docs = {}
for i in file:
    d_docs[i[2]] = i[3]

In [61]:
d_docs

{'что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад': '0',
 'как повысить скорость интернета путем взлома через dns': '0',
 'найти остаток, когда математика 23 ^ 24 математика разделена на 24 23': '0',
 'какая рыба выживет в соленой воде': '0',
 'Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне': '1',
 'что делает детей активными и далеки от телефонных и видеоигр': '0',
 'что я должен делать, чтобы быть великим геологом?': '1',
 'когда вы используете вместо': '0',
 'как я могу взломать motorola dcx3400 для бесплатного интернета': '0',
 'что некоторые технические специалисты могут рассказать о долговечности и надежности ноутбуков и их компонентов': '0',
 'как я могу увидеть все мои комментарии к YouTube': '1',
 'как вы можете легко научиться физике': '1',
 'какой был ваш первый сексуальный опыт': '1',
 'каковы законы об изменении вашего статуса от студенческой визы до зеленой карты в нас, как они сравниваются с иммиграционными закона

In [8]:
docs = []
for i in file:
    docs.append([i[2]])

In [19]:
# корпус уже собран и хранится в json файле
#corpus = []
#for i in docs:
#    j = ' '.join(clean_text(i))
#    corpus.append(j)

In [20]:
#with open("doccorpus.json", "w", encoding = 'utf-8') as dc: ## чтоб не собирать корпус каждый раз заново
#    json.dump(corpus, dc)

In [9]:
with open("doccorpus.json", "r", encoding = 'utf-8') as dc:
    copy_corpus = json.load(dc)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [11]:
corpusik = copy_corpus[:10000] ## корпус слишком большой

In [12]:
X = vectorizer.fit_transform(corpusik)
f_matrix = X.toarray()

In [13]:
## Создание матрицы tf-ов
doc_matrix = np.transpose(f_matrix)
doc_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
## Задание 1. Первый вариант bm25
def bm25(all_n, docs_len, doc_matrix, q_list, corpus, doc, doc_words, k, b, avgdl, N) -> float:
    output = 0
    for word in q_list:
        if word in doc_words:
            w_idx = doc_words.index(word)
            d_idx = corpus.index(doc)
            TF = doc_matrix[w_idx][d_idx]
            n = all_n[word]
            IDF = log((N-n+0.5)/(n+0.5))          
            l_d = docs_len[doc]
            output += IDF * (((TF * (k+1))/(TF + k * (1 - b + (b * (l_d/avgdl))))))
    return output

In [37]:
# Нужно запустить чтоб передать аргументы функциям
doc_words = vectorizer.get_feature_names()
all_n = {}
for word in doc_words:
    w_idx = doc_words.index(word)
    all_n[word] = np.count_nonzero(doc_matrix[w_idx])
    
docs_len = {}
whole_len = 0
for doc in corpusik:
    doc_len = len(doc.split())
    docs_len[doc] = doc_len
whole_len += doc_len

N = len(corpusik) 
avgdl = whole_len/N

In [95]:
## Задание 1. Первый вариант bm25 (функция поиска)
def long_search(query, doc_words, all_n, avgdl, docs_len, N):
    k = 2.0
    b = 0.75     
    
    q_list = [pmm.normal_forms(x)[0] for x in query.split()] 
    docs_bm = []
    for doc, first_doc in zip(corpusik, docs):
        bm = bm25(all_n, docs_len, doc_matrix, q_list, corpusik, doc, doc_words, k, b, avgdl, N)
        docs_bm.append((first_doc, bm))
    response = sorted(docs_bm,key=itemgetter(1), reverse = True)
    print(response[:10])
    return response

In [40]:
## Задание 1. Первый вариант bm25 (смотрим время работы на 1 запросе - даже на одном как-то очень долго)
start_time = time.time()
long_search('где я могу найти американскую базу данных семейного офиса', doc_words, all_n, avgdl, docs_len, N)
print("--- %s seconds ---" % (time.time() - start_time))

[(['где я могу найти американскую базу данных семейного офиса'], 0.005641341788425693), (['пить более 6 литров воды в день, хорошо или плохо для здоровья'], 0.004910645338685187), (['вы пишете взять или сдавать экзамен?'], 0.002387108138932844), (['как я могу получить больше приклада, естественно'], 0.0021918701768153344), (['что вы думаете о запрете на 500 и 1000 наименований в Индии?'], 0.0020253541018345595), (['какова была ваша самая большая проблема в качестве государственного секретаря'], 0.0020200362978866006), (['как преобразовать необработанные файлы в jpeg'], 0.0020200362978866006), (['сколько способов мы можем создать объект в java'], 0.0020200362978866006), (['как можно зарабатывать деньги в tutoruniverse com'], 0.0020200362978866006), (['пунктуация: правильно ли добавить вопросительный знак после риторического вопроса'], 0.0019722016409917226)]
--- 32.27699589729309 seconds ---


In [47]:
## Задание 1. Второй вариант bm25
def matr_bm25(all_n, docs_len, doc_matrix, corpus, doc_words, avgdl, N):
    k = 2.0
    b = 0.75
    bm_matrix = []
    for idx, word in enumerate(doc_words):
        bm = 0
        bm_matrix.append([])
        for doc in corpus:
            if word in doc:
                w_idx = doc_words.index(word)
                d_idx = corpus.index(doc)
                TF = doc_matrix[w_idx][d_idx]
                n = all_n[word]
                IDF = log((N-n+0.5)/(n+0.5))          
                l_d = docs_len[doc]
                bm = IDF * ((TF * (k+1))/(TF + k * (1 - b + (b * (l_d/avgdl)))))
            bm_matrix[idx].append(bm)
    return bm_matrix      

In [48]:
## Задание 1. Второй вариант bm25 (функция создания вектора из запроса)
def vect_bm25(query, k, b, doc_words):
    vect = []
    IDF = 1.0986122886681096 ## посчитала отдельно
    q_words = [pmm.normal_forms(x)[0] for x in query.split()]
    for word in doc_words:
        bm = 0
        if word in q_words:
            l_d = len(q_words)
            TF = 1/l_d
            bm = IDF * ((TF * (k+1))/(TF + k))
        vect.append(bm)
    vect = np.array(vect)
    return vect

In [50]:
## получаем матрицу bm'ов для корпуса
bm_matrix = matr_bm25(all_n, docs_len, doc_matrix, corpusik, doc_words, avgdl, N)
bm_matrix = np.array(bm_matrix)

In [51]:
bm_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00070436, 0.00070436,
        0.00070436],
       [0.        , 0.        , 0.        , ..., 0.001364  , 0.001364  ,
        0.001364  ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [96]:
## Задание 1. Второй вариант bm25 (функция поиска)
def search(query, bm_matrix):
    doc_words = vectorizer.get_feature_names()
    k = 2.0
    b = 0.75
    q_vect = vect_bm25(query, k, b, doc_words)
    doc_score = q_vect.dot(bm_matrix)
    response = list(zip(docs, doc_score))
    response = sorted(response,key=itemgetter(1), reverse = True)
    print(response[:10])
    return response

In [53]:
## Задание 1. Второй вариант bm25 (смотрим время работы на 1 запросе) 
## Гораздо быстрее, чем первый, но зато совсем неправильно...... как же так
start_time = time.time()
search('где я могу найти американскую базу данных семейного офиса', bm_matrix)
print("--- %s seconds ---" % (time.time() - start_time))

[(['где бог'], 0.0013568054219343087), (['почему в некоторых ответах нет комментариев'], 0.0012854705085903935), (['какие гарниры хорошо сочетаются с чечевичным супом'], 0.0012854705085903935), (['какие лучшие доступные гаджеты технологий'], 0.0012854705085903935), (['как можно было бы очистить космический мусор на земной орбите'], 0.0012854705085903935), (['вы пишете взять или сдавать экзамен?'], 0.0012793224292901955), (['как может иностранец получить работу в Южной Корее'], 0.0012793224292901955), (['создает спутниковые блоги и публикует seo-friendly сообщения, которые рассматриваются в технике black-hat seo'], 0.0012793224292901955), (['возможно ли путешествовать быстрее, чем свет'], 0.0011759610910318755), (['каково это быть пожарным'], 0.0011759610910318755)]
--- 0.3712778091430664 seconds ---


In [54]:
## ЗАДАНИЕ 2.
search('рождественские каникулы', bm_matrix)

[(['в охотнике x охотник, который, должно быть, послал орла, чтобы напасть на комуги'], 0.0016398744284181227), (['что является лучшим вариантом для инвестиций'], 0.0016398744284181227), (['почему в голландской империи было так мало колоний в XX веке'], 0.0016398744284181227), (['является хамасом террористической группы'], 0.0016398744284181227), (['я должен уделить время от знакомства, чтобы сосредоточиться на изменении карьеры. Я 33-летний мужчина'], 0.0016398744284181227), (['что должен делать парень, когда в середине разговора его девушка переключается на просмотр фильмов, чем разговаривает с ним'], 0.0016398744284181227), (['почему мой ноутбук перегревается'], 0.0016398744284181227), (['я страдаю от пограничного расстройства личности, и мой парень не понимает его, как я могу помочь ему понять, поэтому он будет более терпеливым со мной'], 0.0016398744284181227), (['как вы справляетесь с работой, когда вы и ваша девушка работаете в одном офисе'], 0.0016398744284181227), (['рабочие, 

[(['в охотнике x охотник, который, должно быть, послал орла, чтобы напасть на комуги'],
  0.0016398744284181227),
 (['что является лучшим вариантом для инвестиций'], 0.0016398744284181227),
 (['почему в голландской империи было так мало колоний в XX веке'],
  0.0016398744284181227),
 (['является хамасом террористической группы'], 0.0016398744284181227),
 (['я должен уделить время от знакомства, чтобы сосредоточиться на изменении карьеры. Я 33-летний мужчина'],
  0.0016398744284181227),
 (['что должен делать парень, когда в середине разговора его девушка переключается на просмотр фильмов, чем разговаривает с ним'],
  0.0016398744284181227),
 (['почему мой ноутбук перегревается'], 0.0016398744284181227),
 (['я страдаю от пограничного расстройства личности, и мой парень не понимает его, как я могу помочь ему понять, поэтому он будет более терпеливым со мной'],
  0.0016398744284181227),
 (['как вы справляетесь с работой, когда вы и ваша девушка работаете в одном офисе'],
  0.00163987442841

In [56]:
long_search('рождественские каникулы', doc_words, all_n, avgdl, docs_len, N) ## А тут одинаково странная выдача для обоих поисков...

[(['в охотнике x охотник, который, должно быть, послал орла, чтобы напасть на комуги'], 0.0024877967191473988), (['каковы меры предосторожности при работе с дробовиками, предлагаемыми nra в massachusetts'], 0.0009047397009818628), (['что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад'], 0.0), (['как повысить скорость интернета путем взлома через dns'], 0.0), (['найти остаток, когда математика 23 ^ 24 математика разделена на 24 23'], 0.0), (['какая рыба выживет в соленой воде'], 0.0), (['Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне'], 0.0), (['что делает детей активными и далеки от телефонных и видеоигр'], 0.0), (['что я должен делать, чтобы быть великим геологом?'], 0.0), (['когда вы используете вместо'], 0.0)]


[(['в охотнике x охотник, который, должно быть, послал орла, чтобы напасть на комуги'],
  0.0024877967191473988),
 (['каковы меры предосторожности при работе с дробовиками, предлагаемыми nra в massachusetts'],
  0.0009047397009818628),
 (['что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад'],
  0.0),
 (['как повысить скорость интернета путем взлома через dns'], 0.0),
 (['найти остаток, когда математика 23 ^ 24 математика разделена на 24 23'],
  0.0),
 (['какая рыба выживет в соленой воде'], 0.0),
 (['Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне'],
  0.0),
 (['что делает детей активными и далеки от телефонных и видеоигр'], 0.0),
 (['что я должен делать, чтобы быть великим геологом?'], 0.0),
 (['когда вы используете вместо'], 0.0)]

Ниже Задание 3.

In [93]:
def long_search_bm15(query, doc_words, all_n, avgdl, docs_len, N):
    k = 2.0
    b = 0.75     
    
    q_list = [pmm.normal_forms(x)[0] for x in query.split()] 
    docs_bm = []
    for doc, first_doc in zip(corpusik, docs):
        bm = bm25(all_n, docs_len, doc_matrix, q_list, corpusik, doc, doc_words, k, b, avgdl, N)
        docs_bm.append((first_doc, bm))
    response = sorted(docs_bm,key=itemgetter(1), reverse = True)
    print(response[:10])
    return response

In [94]:
def long_search_bm11(query, doc_words, all_n, avgdl, docs_len, N):
    k = 2.0
    b = 0.75     
    
    q_list = [pmm.normal_forms(x)[0] for x in query.split()] 
    docs_bm = []
    for doc, first_doc in zip(corpusik, docs):
        bm = bm25(all_n, docs_len, doc_matrix, q_list, corpusik, doc, doc_words, k, b, avgdl, N)
        docs_bm.append((first_doc, bm))
    response = sorted(docs_bm,key=itemgetter(1), reverse = True)
    print(response[:10])
    return response

In [98]:
response = long_search('где я могу найти американскую базу данных семейного офиса', doc_words, all_n, avgdl, docs_len, N)
good_responces = 0
for d in response:
    if int(d_docs[d[0][0]]) == 1:
        good_responces += 1
P = good_responces/len(response)
print('Точность BM25 = ', P) 

[(['где я могу найти американскую базу данных семейного офиса'], 0.005641341788425693), (['пить более 6 литров воды в день, хорошо или плохо для здоровья'], 0.004910645338685187), (['вы пишете взять или сдавать экзамен?'], 0.002387108138932844), (['как я могу получить больше приклада, естественно'], 0.0021918701768153344), (['что вы думаете о запрете на 500 и 1000 наименований в Индии?'], 0.0020253541018345595), (['какова была ваша самая большая проблема в качестве государственного секретаря'], 0.0020200362978866006), (['как преобразовать необработанные файлы в jpeg'], 0.0020200362978866006), (['сколько способов мы можем создать объект в java'], 0.0020200362978866006), (['как можно зарабатывать деньги в tutoruniverse com'], 0.0020200362978866006), (['пунктуация: правильно ли добавить вопросительный знак после риторического вопроса'], 0.0019722016409917226)]
Точность BM25 =  0.3706


In [99]:
response = search('где я могу найти американскую базу данных семейного офиса', bm_matrix)
good_responces = 0
for d in response:
    if int(d_docs[d[0][0]]) == 1:
        good_responces += 1
P = good_responces/len(response)
print('Точность BM25 = ', P) ## так же как быстрый бм25 и бм11

[(['где бог'], 0.0013568054219343087), (['почему в некоторых ответах нет комментариев'], 0.0012854705085903935), (['какие гарниры хорошо сочетаются с чечевичным супом'], 0.0012854705085903935), (['какие лучшие доступные гаджеты технологий'], 0.0012854705085903935), (['как можно было бы очистить космический мусор на земной орбите'], 0.0012854705085903935), (['вы пишете взять или сдавать экзамен?'], 0.0012793224292901955), (['как может иностранец получить работу в Южной Корее'], 0.0012793224292901955), (['создает спутниковые блоги и публикует seo-friendly сообщения, которые рассматриваются в технике black-hat seo'], 0.0012793224292901955), (['возможно ли путешествовать быстрее, чем свет'], 0.0011759610910318755), (['каково это быть пожарным'], 0.0011759610910318755)]
Точность BM25 =  0.3706


In [100]:
response = long_search_bm15('где я могу найти американскую базу данных семейного офиса', doc_words, all_n, avgdl, docs_len, N)
good_responces = 0
for d in response:
    if int(d_docs[d[0][0]]) == 1:
        good_responces += 1
P = good_responces/len(response)
print('Точность BM15 = ', P) 

[(['где я могу найти американскую базу данных семейного офиса'], 0.005641341788425693), (['пить более 6 литров воды в день, хорошо или плохо для здоровья'], 0.004910645338685187), (['вы пишете взять или сдавать экзамен?'], 0.002387108138932844), (['как я могу получить больше приклада, естественно'], 0.0021918701768153344), (['что вы думаете о запрете на 500 и 1000 наименований в Индии?'], 0.0020253541018345595), (['какова была ваша самая большая проблема в качестве государственного секретаря'], 0.0020200362978866006), (['как преобразовать необработанные файлы в jpeg'], 0.0020200362978866006), (['сколько способов мы можем создать объект в java'], 0.0020200362978866006), (['как можно зарабатывать деньги в tutoruniverse com'], 0.0020200362978866006), (['пунктуация: правильно ли добавить вопросительный знак после риторического вопроса'], 0.0019722016409917226)]
Точность BM15 =  0.3706


In [101]:
response = long_search_bm11('где я могу найти американскую базу данных семейного офиса', doc_words, all_n, avgdl, docs_len, N)
good_responces = 0
for d in response:
    if int(d_docs[d[0][0]]) == 1:
        good_responces += 1
P = good_responces/len(response)
print('Точность BM15 = ', P)

[(['где я могу найти американскую базу данных семейного офиса'], 0.005641341788425693), (['пить более 6 литров воды в день, хорошо или плохо для здоровья'], 0.004910645338685187), (['вы пишете взять или сдавать экзамен?'], 0.002387108138932844), (['как я могу получить больше приклада, естественно'], 0.0021918701768153344), (['что вы думаете о запрете на 500 и 1000 наименований в Индии?'], 0.0020253541018345595), (['какова была ваша самая большая проблема в качестве государственного секретаря'], 0.0020200362978866006), (['как преобразовать необработанные файлы в jpeg'], 0.0020200362978866006), (['сколько способов мы можем создать объект в java'], 0.0020200362978866006), (['как можно зарабатывать деньги в tutoruniverse com'], 0.0020200362978866006), (['пунктуация: правильно ли добавить вопросительный знак после риторического вопроса'], 0.0019722016409917226)]
Точность BM15 =  0.3706


In [104]:
def search_bm15(query, bm_matrix):
    doc_words = vectorizer.get_feature_names()
    k = 2.0
    b = 0
    q_vect = vect_bm25(query, k, b, doc_words)
    doc_score = q_vect.dot(bm_matrix)
    response = list(zip(docs, doc_score))
    response = sorted(response,key=itemgetter(1), reverse = True)
    return response

In [105]:
def search_bm11(query, bm_matrix):
    doc_words = vectorizer.get_feature_names()
    k = 2.0
    b = 1
    q_vect = vect_bm25(query, k, b, doc_words)
    doc_score = q_vect.dot(bm_matrix)
    response = list(zip(docs, doc_score))
    response = sorted(response,key=itemgetter(1), reverse = True)
    return response

In [106]:
response = search_bm15('где я могу найти американскую базу данных семейного офиса', bm_matrix)
good_responces = 0
for d in response:
    if int(d_docs[d[0][0]]) == 1:
        good_responces += 1
P = good_responces/len(response)
print('Точность быстрого BM15 = ', P) ## так же как быстрый бм25 и бм11

Точность быстрого BM15 =  0.3706


In [107]:
response = search_bm11('где я могу найти американскую базу данных семейного офиса', bm_matrix)
good_responces = 0
for d in response:
    if int(d_docs[d[0][0]]) == 1:
        good_responces += 1
P = good_responces/len(response)
print('Точность быстрого BM15 = ', P) ## так же как быстрый бм25 и бм15

Точность быстрого BM15 =  0.3706


Везде одинаковая точность